# 005: Cross Validate Models

In [1]:
import os
import sys

import numpy as np

sys.path.append("../")
from models import OrdinaryLeastSquares, LogisticRegression, LinearSVM, KNearestNeighbors
from model_selection import cross_validation
from preprocessing import preprocess

%load_ext autoreload
%autoreload 2

def print_results(cv_results):
    print(f"Average F1-score: {np.mean(cv_results.f1_scores)*100:.1f}% ± {np.std(cv_results.f1_scores)*100:.1f}")
    print(f"Average F2-score: {np.mean(cv_results.f2_scores)*100:.1f}% ± {np.std(cv_results.f2_scores)*100:.1f}")
    print(f"Average AUC-ROC: {np.mean(cv_results.auc_rocs)*100:.1f}% ± {np.std(cv_results.auc_rocs)*100:.1f}")
    print(f" & {np.mean(cv_results.f1_scores)*100:.1f}±{np.std(cv_results.f1_scores)*100:.1f} & "
          f"{np.mean(cv_results.f2_scores)*100:.1f}±{np.std(cv_results.f2_scores)*100:.1f} & "
          f"{np.mean(cv_results.auc_rocs)*100:.1f}±{np.std(cv_results.auc_rocs)*100:.1f}")

In [ ]:
x_train, _, y_train, _ = preprocess(one_hot_encoding=True, save_dir="../data/dataset_prep")
x_train.shape

Loading raw data...
Replacing missing value codes with np.nan...
Saving preprocessed data to ../data/dataset_prep...


(328135, 321)

In [2]:
train = np.load("../data/dataset_prep/train.npz")
x_train, y_train = train["x_train"], train["y_train"]
x_train.shape

(328135, 321)

In [2]:
x_train, _, y_train, *_ = preprocess()

Loading raw data...
Replacing missing value codes with np.nan...
Applying one-hot encoding...


In [ ]:
model_settings = [
    #{"model_class": OrdinaryLeastSquares},
    {"model_class": LogisticRegression, "gamma": 1e-1},
    #{"model_class": LinearSVM},
    #{"model_class": KNearestNeighbors, "use_pca": True},
]
for model in model_settings:
    print(f"Cross-validating model: {model['model_class'].__name__}")
    num_samples = int(1e6) if model['model_class'] != KNearestNeighbors else int(1e4)
    cv_results = cross_validation(x_train[:num_samples], y_train[:num_samples], verbose=True, **model)
    print_results(cv_results)
    if num_samples != 1e6: break
    out_dir = "../results"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{model['model_class'].__name__}.txt")

    with open(out_path, "w") as f:
        f.write(f"Model: {model['model_class'].__name__}\n\n")
        f.write("cv_results repr:\n")
        f.write(repr(cv_results))
        f.write("\n\n")
        f.write("Aggregated metrics:\n")
        f.write(f"Average F1-score: {np.mean(cv_results.f1_scores)*100:.1f}% ± {np.std(cv_results.f1_scores)*100:.1f}\n")
        f.write(f"Average F2-score: {np.mean(cv_results.f2_scores)*100:.1f}% ± {np.std(cv_results.f2_scores)*100:.1f}\n")
        f.write(f"Average AUC-ROC: {np.mean(cv_results.auc_rocs)*100:.1f}% ± {np.std(cv_results.auc_rocs)*100:.1f}\n")
    

Cross-validating model: LogisticRegression
Starting fold 1/5 with 262508 samples
Evaluating lambda=0
Condition number of X^T X: 5.23e+16
Iter    0: loss=0.6931
Iter   10: loss=0.6682
Iter   20: loss=0.6449
Iter   30: loss=0.6214
Iter   40: loss=0.6014
Iter   50: loss=0.5848
Iter   60: loss=0.5710
Iter   70: loss=0.5593
Iter   80: loss=0.5495
Iter   90: loss=0.5411
Iter  100: loss=0.5339
Iter  110: loss=0.5277
Iter  120: loss=0.5223
Iter  130: loss=0.5176
Iter  140: loss=0.5135
Iter  150: loss=0.5099
Iter  160: loss=0.5067
Iter  170: loss=0.5038
Iter  180: loss=0.5013
Iter  190: loss=0.4990
Iter  200: loss=0.4970
Iter  210: loss=0.4951
Iter  220: loss=0.4934
Iter  230: loss=0.4919
Iter  240: loss=0.4906
Iter  250: loss=0.4893
Iter  260: loss=0.4882
Iter  270: loss=0.4871
Iter  280: loss=0.4862
Iter  290: loss=0.4853
Iter  300: loss=0.4845
Iter  310: loss=0.4838
Iter  320: loss=0.4831
Iter  330: loss=0.4824
Iter  340: loss=0.4819
Iter  350: loss=0.4813
Iter  360: loss=0.4808
Iter  370: l